In [ ]:
import pandas as pd
import os

In [ ]:
label = pd.read_csv("../data/2025_ADRD_case_finding/data/adrd_study_700_label.csv")
corrected_label = pd.read_csv("corrected_labels.csv")

Relabel the performance after correcting human labels

In [ ]:
label_map = {}
for_df = {'empi': [], 'label': []}
for index, row in corrected_label.iterrows():
    action = row['Actions']
    correct_label = row['label']
    if not pd.isnull(action):
        if "change" in action:
            if correct_label == "yes":
                correct_label = "no"
            else:
                correct_label = "yes"
        label_map[row['empi']] = correct_label
    for_df['empi'].append(row['empi'])
    for_df['label'].append(correct_label)

print(label_map)

In [ ]:
# save_corrected = pd.DataFrame(for_df)
# print(save_corrected)
# save_corrected.to_csv("corrected_everything.csv")

In [ ]:
for file in sorted(os.listdir("results")):
    tp = 2
    fp = 0
    tn = 8
    fn = 0
    df = pd.read_csv(f"results/{file}")
    print(f"Processing {file}")
    for index, row in df.iterrows():
        if row['empi'] in label_map:
            actual_label = label_map[row['empi']]
            df.loc[index, 'label'] = actual_label
            predicted_label = row['prediction']

            if actual_label == "yes":
                if predicted_label == "yes":
                    df.loc[index, 'result'] = 'tp'
                    tp += 1
                else:
                    df.loc[index, 'result'] = 'fn'
                    fn += 1
            else:
                if predicted_label == "yes":
                    df.loc[index, 'result'] = 'fp'
                    fp += 1
                else:
                    df.loc[index, 'result'] = 'tn'
                    tn += 1
        else:
            actual_label = row['label']
            predicted_label = row['prediction']

            if actual_label == "yes":
                if predicted_label == "yes":
                    df.loc[index, 'result'] = 'tp'
                    tp += 1
                else:
                    df.loc[index, 'result'] = 'fn'
                    fn += 1
            else:
                if predicted_label == "yes":
                    df.loc[index, 'result'] = 'fp'
                    fp += 1
                else:
                    df.loc[index, 'result'] = 'tn'
                    tn += 1

        df['prompt_len'] = df['prompt'].str.len()

    df.to_csv(f"results/{file}", index=False)
    print(f"File {file} stats in order of tp, fp, tn, fn is: {tp}\t{fp}\t{tn}\t{fn}")

In [ ]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-4o")

def txt_to_token(text):
    return len(enc.encode(text))

df67 = pd.read_csv("results/67.csv")

# df67['tokens'] = df67['prompt'].apply(txt_to_token)

print(df67['retrieval_time (s)'].describe())
print(df67['generation_time (s)'].describe())